In [1]:
# This script is for assigning the mutation category to mutations in MAF file
# Author: Yiyun
# Date: Jul-21-2020
from optparse import OptionParser
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import re
import time
import os
import pickle
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
### Directory
dir_maf = '../maf_raw/maf_split'  ### splitted maf file in bash 
dir_out = '../maf_out/maf_ind_from_raw'
dir_rf = '../anno_ref/'

# dict_record = pickle.load(open('../proc_09152020/dict_record_new.pkl','rb'))
# If all patients exisits
lp = pickle.load(open('../anno_ref/list_all_patients.pkl','rb'))

### Split individual maf files

In [35]:
### save individual patient files
def split_patient(fsplit):
    df_maf = pd.read_csv(os.path.join(dir_maf,fsplit), sep = '\t', header = None)
    lpatient = df_maf.iloc[:,12].unique()
    for patient in tqdm(lpatient):
        outpath = os.path.join(dir_out,patient+'.to_merge.csv')
        df_patient = df_maf[df_maf.iloc[:,12] == patient]
        if os.path.exists(outpath):
            df_patient.to_csv(outpath, mode ='a', header = None)
        else:
            df_patient.to_csv(outpath)

# lmaf_split = os.listdir(dir_maf)
# p = Pool(14)
# p.map(split_patient, lmaf_split)
# p.close()
# p.join()

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,28,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,28,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
  0%

 22%|██▏       | 9/41 [00:01<00:06,  4.62it/s]s]/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
 96%|█████████▌| 24/25 [00:23<00:00,  1.96it/s]]/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
 94%|█████████▍| 100/106 [00:28<00:02,  2.19it/s]/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,28,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarn

### Assign categ to individual patients

In [3]:
### directory
dir_out = '../maf_out/maf_ind_from_raw'
dir_categ_out = '../maf_out/maf_ind'
maf_cols = pickle.load(open('../anno_ref/maf_cols.pkl', 'rb')) #maf columns 

In [4]:
# Global variable nucleotides and condon table
nuc = ["A", "T", "C", "G"]
table = {
    'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
    'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
    'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
    'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
    'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
    'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
    'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
    'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
    'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
    'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
    'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
    'TAC': 'Y', 'TAT': 'Y', 'TAA': '_', 'TAG': '_',
    'TGC': 'C', 'TGT': 'C', 'TGA': '_', 'TGG': 'W',
}

gc_di_nuc = "CG"  # definition of CG dinucleotide, from 5' to 3'
stop_codon = ["TAA", "TAG", "TGA"]
null_list = ["Nonstop_Mutation", "Nonsense_Mutation", "Splice_Site", "DEL", "INS"]

# this function determines transition mutation
def is_transition(ori_allele, alt_allele):
    p = False
    transition = [("A", "G"), ("G", "A"), ("T", "C"), ("C", "T")]
    transversion = [("C", "A"), ("C", "G"), ("G", "T"), ("T", "G"), ("G", "C"), ("A", "C"), ("A", "T"), ("T", "A")]
    if (ori_allele, alt_allele) in transition:
        p = True
    elif (ori_allele, alt_allele) in transversion:
        p = False
    else:
#         print("NEITHER TRANSITION NOR TRANSVERSION MUTATION")
        pass

    return p


# this function calculate the contribution of the coverage of a certain base in CpG or not to different categs
def validate_categ(original_allele, altered_allele,
                   tri_nucleotide):  # This before and after base is not same with tri-nucleotide context. It's not in codon context.
    base_before = tri_nucleotide[0]
    base_after = tri_nucleotide[2]

    # Initialize the categ fractions
    categ = 0
    # If a C:G basepairs mutation
    # If mutation in CpG dinucleotide
    if original_allele == "A" or original_allele == "T":
        if is_transition(original_allele, altered_allele):
            categ = 5
        else:
            categ = 6
    elif (original_allele + base_after == gc_di_nuc) or (
            base_before + original_allele == gc_di_nuc):  # This is a CG dinucleotide
        if is_transition(original_allele, altered_allele):
            categ = 1
        else:
            categ = 2
    else:
        if is_transition(original_allele, altered_allele):
            categ = 3
        else:
            categ = 4

    return categ

In [5]:
def categ_assign(patient):
    df_maf = pd.read_csv(os.path.join(dir_out,patient+'.to_merge.csv'), index_col = 0)
    df_maf = df_maf.reset_index(drop = True)
    df_maf.columns = maf_cols

    for idx in df_maf.index:
        genome_change = df_maf.loc[idx,'Genome_Change']
        ref_contxt = df_maf.loc[idx,'ref_context']
        if '>' in genome_change:
            alleles = re.findall(r'[A-Z]+>[A-Z]+', genome_change)[0].split('>')
            ref_allele = alleles[0]
            alter_allele = alleles[1]
            mid_pos = int((len(ref_contxt)+1)/2)
            bbase = ref_contxt[mid_pos-1]
            abase = ref_contxt[mid_pos+1]
            tri_nuc = bbase+ref_allele+abase
            categ = validate_categ(ref_allele, alter_allele, tri_nucleotide=tri_nuc)
        else:
            categ = 7
        df_maf.loc[idx,'categ'] = int(categ)
    
    df_maf.to_csv(os.path.join(dir_categ_out, patient+'.to_merge.categ.csv'))

p = Pool(14)
p.map(categ_assign, lp)
p.close()
p.join()

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,25,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/ac

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/a

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,25,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,18,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,25,29,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (18,25,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,34,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (2,25,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anacon